# Link Analysis I

In [3]:
import pandas as pd

files = [
    r"C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-approval-date-1990-1999.csv",
    r"C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv",
    r"C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv",
    r"C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-registration-date-from-jan-2015-to-dec-2016.csv",
    r"C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv"
]

# Read columns only
columns_by_file = {
    f: set(pd.read_csv(f, nrows=0).columns)
    for f in files
}

# Reference: first file
ref_file = files[0]
ref_cols = columns_by_file[ref_file]

print(f"Reference file:\n{ref_file}")
print(f"Columns: {sorted(ref_cols)}\n")

for f, cols in columns_by_file.items():
    missing = ref_cols - cols
    extra = cols - ref_cols

    print("-" * 80)
    print(f"File: {f}")

    if not missing and not extra:
        print("✔ Same columns as reference")
    else:
        if missing:
            print(f"Missing columns: {sorted(missing)}")
        if extra:
            print(f"Extra columns: {sorted(extra)}")


Reference file:
C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-approval-date-1990-1999.csv
Columns: ['block', 'flat_model', 'flat_type', 'floor_area_sqm', 'lease_commence_date', 'month', 'resale_price', 'storey_range', 'street_name', 'town']

--------------------------------------------------------------------------------
File: C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-approval-date-1990-1999.csv
✔ Same columns as reference
--------------------------------------------------------------------------------
File: C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-approval-date-2000-feb-2012.csv
✔ Same columns as reference
--------------------------------------------------------------------------------
File: C:\Users\mh\Desktop\HTX\HDB\resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv
✔ Same columns as reference
--------------------------------------------------------------------------------
File: C:\Users\mh\Desktop\HTX\HDB\resale-flat-

In [6]:
import pandas as pd

dfs = []

for f in files:
    df = pd.read_csv(f)

    if "month" in df.columns:
        df = df.rename(columns={"month": "year-month"})
        df[["year", "month"]] = (
            df["year-month"]
            .astype(str)
            .str.split("-", expand=True)
            .astype(int)
        )

    dfs.append(df)

# Combine into a single DataFrame
dfs = pd.concat(dfs, ignore_index=True)


In [5]:
dfs

[       year-month        town  flat_type block       street_name storey_range  \
 0         1990-01  ANG MO KIO     1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
 1         1990-01  ANG MO KIO     1 ROOM   309  ANG MO KIO AVE 1     04 TO 06   
 2         1990-01  ANG MO KIO     1 ROOM   309  ANG MO KIO AVE 1     10 TO 12   
 3         1990-01  ANG MO KIO     1 ROOM   309  ANG MO KIO AVE 1     07 TO 09   
 4         1990-01  ANG MO KIO     3 ROOM   216  ANG MO KIO AVE 1     04 TO 06   
 ...           ...         ...        ...   ...               ...          ...   
 287195    1999-12      YISHUN  EXECUTIVE   611      YISHUN ST 61     10 TO 12   
 287196    1999-12      YISHUN  EXECUTIVE   324       YISHUN CTRL     01 TO 03   
 287197    1999-12      YISHUN  EXECUTIVE   392      YISHUN AVE 6     07 TO 09   
 287198    1999-12      YISHUN  EXECUTIVE   356    YISHUN RING RD     04 TO 06   
 287199    1999-12      YISHUN  EXECUTIVE   358    YISHUN RING RD     01 TO 03   
 
         floor